### Tuples as Data Structures

__Tuples vs. Lists vs. Strings__

| Tuples              | Lists                 | Strings            |
| ------------------- | --------------------- | ------------------ |
| container           | container             | container          |
| order matters       | order matters         | order matters      |
| hetero*/homogeneous | hetero/homogeneous*   | homogeneous        |
| indexable           | indexable             | indexable          |
| iterable            | iterable              | iterable           |
| immutable           | mutable               | immutable          |
| fixed length/order  | variable length/order | fixed length/order |

The immutability of tuples works well for representing data structures, as we can assign meaning to the position of data.

e.g. Circle: (0, 0, 10) or a City ('London', 'UK', 8_780_000)

__Tuples as Data Records__

Because tuples are immutable, we are guaranteed that the data and the data structure will never change.

In [3]:
london = ('London', 'UK', 8_780_000)
new_york = ('New York', 'USA', 8_500_000)

__Extracting Data from Tuples__

In [13]:
city, country, pop = london

In [18]:
# Note how the tuples themselves are heterogeneous, but the list of them is homogeneous
cities = [london, new_york]

total_pop = 0
for city in cities:
    total_pop += city[2]

__Dummy Variables__

An underscore, `_`, can be used to indicate a variable is not intended to be used, or that it can be ignored. This is still a valid variable name however, and still holds the value of whatever was assigned to it. It is simply a convention.

In [20]:
city, _, pop = ('Beijing', 'China', 21_000_000)

In [24]:
# Dummy variables can be used with extended unpacking as well
stock_record = ('DIJA', 2018, 1, 19, 25987.35, 26071.72, 25942.83, 26071.72)

symbol, year, month, day, *_, close = stock_record

### Named Tuples

If we need some sort of simple named structure for a data record, one might think to encapsulate it as a class, such as: 

In [26]:
class Point2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"Point2D(x={self.x}, y={self.y})"
    
    def __eq__(self, other):
        if isinstance(other, Point2D):
            return self.x == other.x and self.y == other.y
        else:
            return False

However, using a class might not be the best approach for simple data structures.

For one thing, the Point2D object is mutable which may not be what we want.

A perceived downside to using tuples is that we lose the labels associated with class properties.

e.g. class: `point.x` vs. a tuple: `point[0]`

But by using __named tuples__, we get the benefits of both classes and tuples!

Named tuples are a subclass of `tuple`, and add a layer to assign property names to positional elements.

In [27]:
from collections import namedtuple

`namedtuple` is a function which generates a new class (a class factory), this new class inherits from `tuple` and provides named properties to access elements of the tuple. But an instacne of this class is still a tuple. 

__Generating Named Tuple Classes__

`namedtuple` needs:
- the class name you want to use
- a sequence of field names to assign, in the order of the elemnts in the tuple
    - field names cannot start with an underscore!
    
The return value of `namedtuple` will be a class, which we will use to construct instances.


In [31]:
# The variable name should be (but not required to be) the same as the one specified in the call to namedtuple().
# And should be capitalized like a class name would be.
Point2D = namedtuple('Point2D', ['x', 'y'])

In [32]:
pt = Point2D(10, 20)

In [33]:
# We can pass the sequence of field names to namedtuple in multiple ways:

Point2D = namedtuple('Point2D', ['x', 'y']) # by list
Point2D = namedtuple('Point2D', ('x', 'y')) # by tuple
Point2D = namedtuple('Point2D', 'x, y' ) # by comma separated string
Point2D = namedtuple('Point2D', 'x y' ) # by space separated string

__Instantiating Named Tuples__

In [34]:
# Via positional arguments
pt = Point2D(10, 20) # x = 10, y = 20

# Via keyword arguments
pt = Point2D(x=10, y=20)

__Accessing Data in a Named Tuple__


In [35]:
x, y = pt

In [36]:
x = pt[0]

In [37]:
for e in pt:
    print(e)

10
20


In [38]:
pt.x

10

In [39]:
pt.y

20

In [40]:
# Since namedtuple generates classes inheriting from Tuple, the class instances are 
# immutable just like normal tuples.
isinstance(pt, tuple)

True

In [41]:
pt.x = 'error'

AttributeError: can't set attribute